# Analyzing OSS Projects under the Apache Software Foundation using GH Archive Data

This notebook aims to analyze the open-source software (OSS) projects using the GH Archive dataset. The goal is to provide insights that can help in making decisions regarding the adoption and exit timing of OSS projects managed by the Apache Software Foundation.

The scope of the analysis will be limited to the following aspects:
1. **Project Owner** - Only projects under the `apache` organization will be considered.
2. **Time Period** - The analysis will only include data from the year 2020 to 2025.


## Data Preparation and Loading

Data relevant to the analysis was queried from BigQuery's `gharchive` public dataset. The results of the query was exported as parquet files on a Google Cloud Storage bucket and copied over the the `/home/ubuntu/lab1/data` folder of the EC2 instance used by the project. 

![Big Query Job Detail Screenshot](./images/bigquery_export.png)

The total data volume of the archive from 2020-2025 is 21.57TB. When filtered to include only the projects handled by the Apache Software Foundation, the volume of the processed data went down 106.91 GB. With compression applied and the file format changed to parquet, the data copied over to the EC2 volume was 22 GB.

In [10]:
!du -sh ./data

22G	./data


In [ ]:
!du -sh ./*